**Churn Dataframe**

This notebook creates a shared churn dataframe. Company Churn is defined as early teminations of licenses  
or license expirations without renewals.

In [2]:
companies = spark.read.format('delta').load("/mnt/silver/cab/companies")
accounts = spark.read.format('delta').load("/mnt/silver/cab/accounts")
licenses = spark.read.format('delta').load("/mnt/silver/cab/licenses")
trips = spark.read.format('delta').load("/mnt/silver/cab/trips")
users = spark.read.format('delta').load("/mnt/silver/cab/users")
form_headers = spark.read.format('delta').load("/mnt/silver/cab/form_headers")
timekeeping_statuses = spark.read.format('delta').load("/mnt/silver/cab/timekeeping_statuses")
order_headers = spark.read.format('delta').load("/mnt/silver/cab/order_headers")
incidents = spark.read.format('delta').load("/mnt/silver/dynamics/incidents")

**Create the churn dataframe for modeling:**
- Find billable Companies.
- Aggregate Licenses to company level.
- Aggregate dynamics tickets.
- Agregate trips.
- Aggregate forms submitted.

**Global variables**  
They are defined outside of this notebook.

In [5]:
# from datetime import datetime

# # --- license deactivation datetime
# pred_datetime = datetime(2020, 7, 21)

# # --- license activation start datetime
# start_dateime = datetime(2019, 1, 1)

# # --- exclude IsSetupComplete = true
# registered_only = False

** Get billable companies **

In [7]:
from pyspark.sql.functions import sum, min, max, col, current_date, udf, collect_list
from pyspark.sql.types import *

billable_companies = companies.join(accounts, companies.AccountId == accounts.BillingAccountId) \
                              .join(licenses, (licenses.AccountId == companies.AccountId) & (licenses.InstanceId == companies.InstanceId)) \
                              .filter(licenses.Billable == True) \
                              .filter(accounts.IsTest == False) \
                              .drop(licenses.CompanyId).drop(licenses.TierId).drop(licenses.InstanceId) \
                              .select(col('CompanyId'), col('InstanceId'), col('CompanyName'), col('TierId').alias('Tier'), col('SetupCompletionTime').alias('IsSetupComplete'), \
                                      col('UserIntegrationType'), col('BillingAccountId').alias('AccountId'), accounts.AccountName, \
                                      col('City'), col('RegionName'), col('PostalCode'), col('CountryCode'), col('Culture')) \
                              .distinct()

billable_companies = billable_companies.withColumn('IsSetupComplete', billable_companies.IsSetupComplete.isNotNull())

# AccountId=2520 has multiple 6 companies.
# display(billable_companies.filter('AccountId=2520'))

# we filter out Republic (accountId 4700 or companyId 5788) that has huge number of licenses.
billable_companies = billable_companies.filter(col('UserIntegrationType').isNotNull()).filter(col('AccountId') != 4700)
# billable_companies = billable_companies.filter(col('UserIntegrationType').isNotNull())

if registered_only == True:
  billable_companies = billable_companies.filter(col('IsSetupComplete') == True)


** Calculate Company Churn **
- add features: licenses by company, license duration. 
- add target: Churn

In [9]:
# group licenses by company based on pred_datetime, excluding Activation > pred_datetime.
licenses_by_account = licenses.groupBy('CompanyId', 'AccountId', 'InstanceId') \
                              .agg(min('CreatedOn').alias('Activation'), max('ExpirationDate').alias('Expiration'), max('DeactDate').alias('Deactivation')) \
                              .filter(col('Activation').isNotNull()) \
                              .filter((col('Activation') >= start_dateime) & (col('Activation') < pred_datetime))

# agg over 'Count' isn't correct. We need to do custom sum() over 'Count'
@udf(returnType=IntegerType())
def getLicenseCounts(vals, licenses, orginalLicenses, statuses):
  sum = 0
  i = 0
  for val in vals:
    # only sum active license counts
    if statuses[i] == 0:
      sum += val
    else:
      # for inactive licenses, only count original licenses 
      if orginalLicenses[i] == 0:
        sum += val
    i += 1
  return sum

# OriginalLicenseId = null if no tier change etc. collect_list will not include those null values in aggregation.
# Fill them with 0 first to facillitate the calculation.
licenses_tmp = licenses.fillna({'OriginalLicenseId' : 0})
license_count = licenses_tmp.groupBy('CompanyId', 'AccountId').agg(getLicenseCounts(collect_list('Count'), collect_list('LicenseId'), collect_list('OriginalLicenseId'), collect_list('Status')).alias('Licenses'))

# calculate Churn or No-Churn
@udf(returnType=IntegerType())
def isChurned(colDeactivation):
  if colDeactivation is not None and colDeactivation < pred_datetime:
    return 1
  return 0

# calculate license duration in days
@udf(returnType=IntegerType())
def licenseDuration(colActivation, colDeactivation):
  if colActivation is not None and colActivation < pred_datetime:
    if colDeactivation is not None and colDeactivation < pred_datetime:
      delta = colDeactivation - colActivation
      # colDeactivation can be smaller than colActivation if it occurs within one day
      # because of the bug in db.
      return 0 if delta.days < 0 else delta.days
    else:
      delta = pred_datetime - colActivation
      return delta.days
  # return -1 for activation is after pred_datetime  
  return -1 

# calculate License Duration, and Churn based on pred_datetime
company_churn = licenses_by_account.withColumn('Duration', licenseDuration(col('Activation'), col('Deactivation'))).withColumn('Churn', isChurned(col('Deactivation')))
# add license counts
company_churn = company_churn.join(license_count, (company_churn.CompanyId == license_count.CompanyId) & (company_churn.AccountId == license_count.AccountId)) \
                             .drop(license_count.AccountId).drop(license_count.CompanyId)

# exclude records with activation after pred_datetime
company_churn = company_churn.filter(company_churn.Duration != -1)
# display(company_churn.filter('CompanyId=23'))

Add feature: incidents by account

In [11]:
# accounts.createOrReplaceTempView('Accounts')
# incidents.createOrReplaceTempView('Incidents')

# cutoff_date = pred_datetime.strftime('%Y-%m-%d')

# account_incidents = spark.sql('''
#                             SELECT 
#                                 a.BillingAccountId as AccountId
#                                 ,a.AccountName
#                                 ,a.Status
#                                 ,(SELECT count(1) FROM Incidents i WHERE a.accountid == i.accountid) as IncidentCount
#                             FROM Accounts a
#                             where a.BillingAccountId is not null and a.IsTest=false
#                       ''')

# display(account_incidents.orderBy(col('IncidentCount').desc()))

In [12]:
account_incidents_tmp = accounts.drop('modifiedon').join(incidents, incidents.accountid == accounts.accountid) \
                            .filter(col('BillingAccountId').isNotNull()) \
                            .filter(col('IsTest') == False) \
                            .filter(col('modifiedon') < pred_datetime)

account_incidents = account_incidents_tmp.groupBy('BillingAccountId').count().withColumnRenamed('count', 'Incidents')

# display(account_incidents.filter('BillingAccountId=2520'))

Add feature: number of trips by company

In [14]:
company_trips = trips.filter(col('StartLocalTime') < pred_datetime) \
                  .groupBy('CompanyId', 'InstanceId').count().withColumnRenamed('count', 'Trips')
                  

Add feature: number of forms submitted by company

In [16]:
company_forms = form_headers.filter(col('StartLocalTimeTag') < pred_datetime) \
                  .groupBy('CompanyId', 'InstanceId').count().withColumnRenamed('count', 'Forms')
                  

Add feature: number of timekeeping submitted by company

In [18]:
timekeeping = timekeeping_statuses.filter(col('StartLocalTime') < pred_datetime) \
                  .groupBy('CompanyId', 'InstanceId').count().withColumnRenamed('count', 'Timekeeping')

Add feature: number of orders submitted by company

In [20]:
orders = order_headers.filter(col('StartTime') < pred_datetime) \
            .groupBy('CompanyId', 'InstanceId').count().withColumnRenamed('count', 'Orders')

** Account Churn: combine all features **

In [22]:
billable_companies_w_licenses = billable_companies.join(company_churn, (company_churn.CompanyId == billable_companies.CompanyId) & (company_churn.InstanceId == billable_companies.InstanceId)) \
                                     .drop(company_churn.CompanyId).drop(company_churn.AccountId).drop(company_churn.InstanceId)

#**** aggregate "incidents". "incidents" only has accountId to be associated with.
billable_account_churn = billable_companies_w_licenses.join(account_incidents, account_incidents.BillingAccountId == billable_companies_w_licenses.AccountId, "left_outer") \
                                    .drop(account_incidents.BillingAccountId)
# fill null as 0 if no incidents
billable_account_churn = billable_account_churn.fillna({'Incidents' : 0})

#**** aggregate "trips"
billable_account_churn = billable_account_churn.join(company_trips, (company_trips.CompanyId == billable_account_churn.CompanyId) & (company_trips.InstanceId == billable_account_churn.InstanceId), "left_outer") \
                                    .drop(company_trips.CompanyId).drop(company_trips.InstanceId)
# fill null as 0 if no Trips
billable_account_churn = billable_account_churn.fillna({'Trips' : 0})

#*** aggregate "forms"
billable_account_churn = billable_account_churn.join(company_forms, (company_forms.CompanyId == billable_account_churn.CompanyId) & (company_forms.InstanceId == billable_account_churn.InstanceId), "left_outer") \
                                    .drop(company_forms.CompanyId).drop(company_forms.InstanceId)
# fill null as 0 if no Forms
billable_account_churn = billable_account_churn.fillna({'Forms' : 0})

#*** aggregate "timekeeping"
billable_account_churn = billable_account_churn.join(timekeeping, (timekeeping.CompanyId == billable_account_churn.CompanyId) & (timekeeping.InstanceId == billable_account_churn.InstanceId), "left_outer") \
                                    .drop(timekeeping.CompanyId).drop(timekeeping.InstanceId)
# fill null as 0 if no Timekeeping
billable_account_churn = billable_account_churn.fillna({'Timekeeping' : 0})

#** aggregate "orders"
billable_account_churn = billable_account_churn.join(orders, (orders.CompanyId == billable_account_churn.CompanyId) & (orders.InstanceId == billable_account_churn.InstanceId), "left_outer") \
                                    .drop(orders.CompanyId).drop(orders.InstanceId)
# fill null as 0 if no Orders
billable_account_churn = billable_account_churn.fillna({'Orders' : 0})
# cache it
billable_account_churn_cache = billable_account_churn.cache()


In [23]:
# display(billable_account_churn_cache.filter('AccountId=4700'))